In [2]:
import pandas as pd
import edhec_risk_ph as erk
%load_ext autoreload
%autoreload 2


In [3]:
returns = erk.get_ffme_returns()

/Users/paulhake/Documents/claude_code_class/notebooks_and_codem01_v02/nb/edhec_risk_ph.py:50: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  me_m = pd.read_csv(file_path, header=0, parse_dates=True, index_col=0, na_values=-99.99)


In [4]:
returns.head()

SmallCap  LargeCap
1926-07   -0.0057    0.0333
1926-08    0.0384    0.0233
1926-09   -0.0048   -0.0009
1926-10   -0.0329   -0.0295
1926-11   -0.0055    0.0316

In [6]:
# Calculate annualized returns for both columns
annualized_returns = erk.annualize_rets(returns, 12)
annualized_returns

SmallCap    0.151977
LargeCap    0.098490
dtype: float64

In [7]:
annualized_vol = erk.annualize_vol(returns, 12)
annualized_vol

SmallCap    0.336701
LargeCap    0.195116
dtype: float64

In [8]:
# Test if slicing is inclusive on both ends for 1999-2015
test_subset = returns["1999":"2015"]
print(f"First date: {test_subset.index[0]}")
print(f"Last date: {test_subset.index[-1]}")
print(f"Total months: {len(test_subset)}")


First date: 1999-01
Last date: 2015-12
Total months: 204


In [12]:
# Calculate annualized return for SmallCap (Lo 20) from 1999-2015
ann_ret_1999_2015 = erk.annualize_rets(test_subset["LargeCap"], 12)
print(f"\nAnnualized Return (Lo 20, 1999-2015): {ann_ret_1999_2015:.4%}")


Annualized Return (Lo 20, 1999-2015): 6.2865%


In [13]:
erk.annualize_vol(test_subset["LargeCap"], 12)

np.float64(0.1727133212125047)

In [16]:
# Calculate max drawdown for SmallCap (Lo 20) from 1999-2015
dd = erk.drawdown(test_subset["LargeCap"])
max_dd = dd["Drawdown"].min()
max_dd_date = dd["Drawdown"].idxmin()

print(f"Max Drawdown: {max_dd:.4%}")
print(f"Date of Max Drawdown: {max_dd_date}")

Max Drawdown: -55.2735%
Date of Max Drawdown: 2009-02


In [14]:
erk.drawdown(test_subset["SmallCap"]).max()

Wealth      7.558245
Peaks       7.558245
Drawdown    0.000000
dtype: float64

### deviations from normality

In [17]:
hfi = erk.get_hfi_returns()
hfi.head()

/Users/paulhake/Documents/claude_code_class/notebooks_and_codem01_v02/nb/edhec_risk_ph.py:75: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  hfi = pd.read_csv(file_path, header=0, parse_dates=True, index_col=0)


Convertible Arbitrage  CTA Global  Distressed Securities  \
date                                                                
1997-01                 0.0119      0.0393                 0.0178   
1997-02                 0.0123      0.0298                 0.0122   
1997-03                 0.0078     -0.0021                -0.0012   
1997-04                 0.0086     -0.0170                 0.0030   
1997-05                 0.0156     -0.0015                 0.0233   

         Emerging Markets  Equity Market Neutral  Event Driven  \
date                                                             
1997-01            0.0791                 0.0189        0.0213   
1997-02            0.0525                 0.0101        0.0084   
1997-03           -0.0120                 0.0016       -0.0023   
1997-04            0.0119                 0.0119       -0.0005   
1997-05            0.0315                 0.0189        0.0346   

         Fixed Income Arbitrage  Global Macro  Long/Short Equity  \
date                                                               
1997-01                  0.0191        0.0573             0.0281   
1997-02                  0.0122        0.0175            -0.0006   
1997-03                  0.0109       -0.0119            -0.0084   
1997-04                  0.0130        0.0172             0.0084   
1997-05                  0.0118        0.0108             0.0394   

         Merger Arbitrage  Relative Value  Short Selling  Funds Of Funds  
date                                                                      
1997-01            0.0150          0.0180        -0.0166          0.0317  
1997-02            0.0034          0.0118         0.0426          0.0106  
1997-03            0.0060          0.0010         0.0778         -0.0077  
1997-04           -0.0001          0.0122        -0.0129          0.0009  
1997-05            0.0197          0.0173        -0.0737          0.0275

In [31]:
pd.concat([hfi.mean(), hfi.median(), hfi.mean()>hfi.median()], axis=1)

0       1      2
Convertible Arbitrage   0.005508  0.0065  False
CTA Global              0.004074  0.0014   True
Distressed Securities   0.006946  0.0089  False
Emerging Markets        0.006253  0.0096  False
Equity Market Neutral   0.004498  0.0051  False
Event Driven            0.006344  0.0084  False
Fixed Income Arbitrage  0.004365  0.0055  False
Global Macro            0.005403  0.0038   True
Long/Short Equity       0.006331  0.0079  False
Merger Arbitrage        0.005356  0.0060  False
Relative Value          0.005792  0.0067  False
Short Selling          -0.001701 -0.0053   True
Funds Of Funds          0.004262  0.0052  False

In [18]:
skewness = erk.skewness(hfi).sort_values(ascending=True)
skewness

Fixed Income Arbitrage   -3.940320
Convertible Arbitrage    -2.639592
Equity Market Neutral    -2.124435
Relative Value           -1.815470
Event Driven             -1.409154
Merger Arbitrage         -1.320083
Distressed Securities    -1.300842
Emerging Markets         -1.167067
Long/Short Equity        -0.390227
Funds Of Funds           -0.361783
CTA Global                0.173699
Short Selling             0.767975
Global Macro              0.982922
dtype: float64

In [36]:
skewness

Global Macro              0.982922
Short Selling             0.767975
CTA Global                0.173699
Funds Of Funds           -0.361783
Long/Short Equity        -0.390227
Emerging Markets         -1.167067
Distressed Securities    -1.300842
Merger Arbitrage         -1.320083
Event Driven             -1.409154
Relative Value           -1.815470
Equity Market Neutral    -2.124435
Convertible Arbitrage    -2.639592
Fixed Income Arbitrage   -3.940320
dtype: float64

In [56]:
from scipy.stats import skew, kurtosis, jarque_bera
skew(hfi)

array([-2.63959223,  0.17369864, -1.30084204, -1.16706749, -2.12443538,
       -1.40915356, -3.94032029,  0.98292188, -0.39022677, -1.32008333,
       -1.81546975,  0.76797484, -0.36178308])

In [50]:
import numpy as np
normal_rets = np.random.normal(0, 0.15, size=(263,1))
skew(normal_rets)

array([-0.09675258])

In [51]:
erk.skewness(normal_rets)

np.float64(-0.09675257697202864)

In [52]:
erk.kurtosis(normal_rets)

np.float64(2.658452985820994)

In [55]:
erk.kurtosis(hfi).sort_values()

CTA Global                 2.952960
Long/Short Equity          4.523893
Global Macro               5.741679
Short Selling              6.117772
Funds Of Funds             7.070153
Distressed Securities      7.889983
Event Driven               8.035828
Merger Arbitrage           8.738950
Emerging Markets           9.250788
Relative Value            12.121208
Equity Market Neutral     17.218555
Convertible Arbitrage     23.280834
Fixed Income Arbitrage    29.842199
dtype: float64

In [57]:
jarque_bera(normal_rets)

SignificanceResult(statistic=np.float64(1.6886639071601837), pvalue=np.float64(0.4298444183401069))

In [61]:
jarque_bera(hfi)

SignificanceResult(statistic=np.float64(25656.585999171337), pvalue=np.float64(0.0))

In [64]:
erk.is_normal(normal_rets)

np.True_

In [69]:
hfi.aggregate(erk.is_normal)

Convertible Arbitrage     False
CTA Global                 True
Distressed Securities     False
Emerging Markets          False
Equity Market Neutral     False
Event Driven              False
Fixed Income Arbitrage    False
Global Macro              False
Long/Short Equity         False
Merger Arbitrage          False
Relative Value            False
Short Selling             False
Funds Of Funds            False
dtype: bool

In [70]:
ffme = erk.get_ffme_returns()
ffme.aggregate(erk.is_normal)

/Users/paulhake/Documents/claude_code_class/notebooks_and_codem01_v02/nb/edhec_risk_ph.py:50: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  rets = me_m[['Lo 10', 'Hi 10']] / 100


SmallCap    False
LargeCap    False
dtype: bool

In [71]:
erk.skewness(ffme).sort_values()

LargeCap    0.233445
SmallCap    4.410739
dtype: float64

In [72]:
erk.kurtosis(ffme)

SmallCap    46.845008
LargeCap    10.694654
dtype: float64